# Use LMM simulation for Normal Response instead of Element Wise Simulation using Expected Fisher Info

Instead of simulating the random vector element by element and then doing this 10,000 times, here we check the robustness of the estimation procedure by using a more standard way to simulate from the LMM model. 

$\beta = \begin{pmatrix} 1 \\ 1\\ 1 \end{pmatrix}$

$n_i = 20$

$V_1 = 1_n 1_n^t, V_2 = I_n$

$\Gamma = 0.1 * V_1 + 0.1 * V_2$

In [50]:
using GLMCopula, Random, LinearAlgebra, GLM

Random.seed!(123)

n = 20  # number of observations
n_i = 100000
#ns = rand(100:300, n) # ni in each observation
ns = [n_i for i in 1:n]
p = 3   # number of mean parameters
m = 2   # number of variance components

d = Normal()
link = IdentityLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, n)
# true parameter values
βtruth = ones(p)
σ2truth = [0.1]
σ02truth = 0.1
for i in 1:n
    ni = ns[i]
    V1 = ones(ni, ni)
    V2 = Matrix(I, ni, ni)
    Ω = σ2truth[1] * V1 + σ02truth * V2
    Ωchol = cholesky(Symmetric(Ω))
    # simulate design matrix
    X = [ones(ni) randn(ni, p-1)]
    # generate mvn response 
    y = X * βtruth + Ωchol.L * randn(ni)
    # add to data
    gcs[i] = GLMCopulaVCObs(y, X, [V1, V2], d, link)
end

gcm = GLMCopulaVCModel(gcs);

In [51]:
@info "Initial point:"
initialize_model!(gcm)
@show gcm.β
fill!(gcm.Σ, 1)
update_Σ!(gcm)
@show inv.(gcm.τ)
@show gcm.Σ
loglikelihood!(gcm, true, true)

gcm.β = [0.9693590019207219, 1.0016760246003946, 1.0019043528575036]
inv.(gcm.τ) = 

┌ Info: Initial point:
└ @ Main In[51]:1


[0.19815847049362176]
gcm.Σ = [0.1559233187105763, 9.513417562232235e-294]


-60576.61708137205

In [52]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, hessian_approximation = "exact"))

gcm.Σ = [0.1559221744644235, 4.6747044246673334e-294]
gcm.Σ = [0.1559210385040016, 2.2970473201064875e-294]
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only up

In [53]:
@show gcm.β
@show inv.(gcm.τ)
@show gcm.Σ
@show gcm.∇β
@show loglikelihood!(gcm, true, true);

gcm.β = [0.9676514371660554, 1.0015629765375502, 1.001892659933846]
inv.(gcm.τ) = [0.19815638467444507]
gcm.Σ = [781.6271821767783, 0.0]
gcm.∇β = [-3.0712448051417596e-7, -1.51969743455993e-8, 3.1291875757233356e-9]
loglikelihood!(gcm, true, true) = -60575.332027610675


In [42]:
d = Normal()
link = IdentityLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, n)
# true parameter values
βtruth = ones(p)
σ2truth = [0.1, 0.1]
σ02truth = 0.1
for i in 1:n
    ni = ns[i]
    V1 = ones(ni, ni)
    V2 = Matrix(I, ni, ni)
    Ω = σ2truth[1] * V1 + σ2truth[1] * V2 + σ02truth * Matrix(I, ni, ni)
    Ωchol = cholesky(Symmetric(Ω))
    # simulate design matrix
    X = [ones(ni) randn(ni, p-1)]
    # generate mvn response 
    y = X * βtruth + Ωchol.L * randn(ni)
    # add to data
    gcs[i] = GLMCopulaVCObs(y, X, [V1, V2], d, link)
end

gcm = GLMCopulaVCModel(gcs);

In [43]:
@info "Initial point:"
initialize_model!(gcm)
@show gcm.β
fill!(gcm.Σ, 1)
update_Σ!(gcm)
@show inv.(gcm.τ)
@show gcm.Σ
loglikelihood!(gcm, true, true)

gcm.β = [0.9975568822797937, 0.9969269046980099, 0.9967615648419103]
inv.(gcm.τ) = [0.28529761650543756]
gcm.Σ = [0.49133549077205774, 5.325259734862797e-39]


┌ Info: Initial point:
└ @ Main In[43]:1


-38775.219962602554

In [44]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, hessian_approximation = "exact"))

gcm.Σ = [0.49133406943640884, 3.7216473049590076e-39]
gcm.Σ = [0.4913326997904034, 2.600932515794415e-39]
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only uppe

In [45]:
@show gcm.β
@show inv.(gcm.τ)
@show gcm.Σ
@show gcm.∇β
@show loglikelihood!(gcm, true, true);

gcm.β = [0.995528933456643, 0.9965231284472669, 0.996777168185698]
inv.(gcm.τ) = [0.28530086985448583]
gcm.Σ = [0.4918854677880372, 5.5478718912012775e-98]
gcm.∇β = [-6.705887773961194e-7, -2.5901263889238635e-7, -3.458682922286016e-7]
loglikelihood!(gcm, true, true) = -38774.873400832585


In [46]:
d = Normal()
link = IdentityLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, n)
# true parameter values
βtruth = ones(p)
σ2truth = [0.1; 0.1]
σ02truth = 0.1
for i in 1:n
    ni = ns[i]
    # set up covariance matrix
    V1 = convert(Matrix, Symmetric([Float64(i * (ni - j + 1)) for i in 1:ni, j in 1:ni])) # a pd matrix
    V1 ./= norm(V1) / sqrt(ni) # scale to have Frobenius norm sqrt(n)
    prob = fill(1/ni, ni)
    V2 = ni .* (Diagonal(prob) - prob * transpose(prob))
    V2 ./= norm(V2) / sqrt(ni) # scale to have Frobenious norm sqrt(n)
    Ω = σ2truth[1] * V1 + σ2truth[2] * V2 + σ02truth * I
    Ωchol = cholesky(Symmetric(Ω))
    # simulate design matrix
    X = [ones(ni) randn(ni, p-1)]
    # generate mvn response 
    y = X * βtruth + Ωchol.L * randn(ni)
    # add to data
    gcs[i] = GLMCopulaVCObs(y, X, [V1, V2], d, link)
end

gcm = GLMCopulaVCModel(gcs);

In [47]:
@info "Initial point:"
initialize_model!(gcm)
@show gcm.β
fill!(gcm.Σ, 1)
update_Σ!(gcm)
@show inv.(gcm.τ)
@show gcm.Σ
loglikelihood!(gcm, true, true)

gcm.β = [0.9964966590866627, 1.0035747845740537, 0.9953699642208286]
inv.(gcm.τ) = 

┌ Info: Initial point:
└ @ Main In[47]:1


[0.21305794812011306]
gcm.Σ = [154.60272603039047, 5.4e-323]


-32794.6451884259

In [48]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, hessian_approximation = "exact"))

gcm.Σ = [282.83384358699607, 5.4e-323]
gcm.Σ = [408.3001456263511, 5.4e-323]
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    o

In [49]:
@show gcm.β
@show inv.(gcm.τ)
@show gcm.Σ
@show gcm.∇β
@show loglikelihood!(gcm, true, true);

gcm.β = [0.9974114048216862, 1.002886035522901, 0.9951976991504529]
inv.(gcm.τ) = [0.21305044615892013]
gcm.Σ = [2598.0572483725537, 2.5e-323]
gcm.∇β = [3.987609922262436e-7, -5.1733993089442265e-8, -1.8329128437244435e-8]
loglikelihood!(gcm, true, true) = -32794.49816847527
